In [ ]:
import numpy as np 
import os
import json
import librosa
import nemo.collections.asr as nemo_asr
import torch
from nemo.collections.asr.metrics.wer import word_error_rate

In [ ]:
paths2audio_files = "/mnt/data/amicorpus/mixheadset_test_wav.list"
paths2audio_files = np.loadtxt(paths2audio_files, dtype='str')
transcription_folder = "/mnt/data/AMI_ASR_transcripts_v2/"

In [ ]:
with open(os.path.join('AMI_test_manifest.json'), 'w') as fp:
    for audio_file in paths2audio_files:
        text_file = os.path.join(transcription_folder, os.path.basename(audio_file).split(".")[0]+".txt")
        with open(text_file, 'r') as file:
            text = file.readlines()[0]

        y, sr = librosa.load(audio_file, sr=16000)
        dur = librosa.get_duration(y=y, sr=sr)

        entry = {'audio_filepath': audio_file, 'duration': dur, 'offset': 0,'text': text}
        fp.write(json.dumps(entry) + '\n')

In [ ]:
# Load model
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name='stt_en_citrinet_1024_gamma_0_25', strict=False)

In [ ]:
with torch.cuda.amp.autocast():
    transcript = asr_model.transcribe(paths2audio_files, batch_size=1)

In [ ]:
# this is so bad but working, will update and submit PR for citrinet inference eval after vacation. 

all_wer_value = 0
for i in range(len(paths2audio_files)):
    text_file = os.path.join(transcription_folder, os.path.basename(paths2audio_files[i]).split(".")[0]+".txt")
    with open(text_file, 'r') as file:
        text = file.readlines()[0]
    hypotheses = transcript[i]
    wer_value = word_error_rate(hypotheses=[hypotheses], references=[text])
    all_wer_value += wer_value
    print(text_file, wer_value)
    print("Hyp:", hypotheses[:200])
    print("Ref:", text[:200])
    print("=======")
    

In [ ]:
# v2
all_wer_value/len(transcript)